Learning how to set up a RNN model using this notebook:
https://www.kaggle.com/prakash711/m5-forecasting-with-tensorflow-bilstm

What I contributed:
* Created a function that prepares the train data for a RNN model
* Created a function that prepares and predicts on a test data
* Added a callback function to the model
* Documented code cell by cell to understand what is happening

In [ ]:
# Imports

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
# Reading in train data

train = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv")

In [ ]:
# Viewing the first 5 observations

train.head()

In [ ]:
# Function that reduces memory

def reduction_mem(df):
    float_cols = [c for c in df if df[c].dtype == 'float64']
    int_cols = [c for c in df if df[c].dtype in ['int64', 'int32']]
    df[float_cols] = df[ float_cols].astype(np.float16)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df

In [ ]:
# Reducing memory of the data 

train = reduction_mem(train)

In [ ]:
# Getting the columns that has column names

day_columns = [col for col in train.columns if 'd_' in col]

In [ ]:
# Getting only sales data

train = train[day_columns]

In [ ]:
# Transposing the data

train = train.T
train.shape

In [ ]:
# Scaling the features using min-max scaler in range 0-1

## Importing MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

## Calling it
sc = MinMaxScaler(feature_range = (0, 1))

## Fitting on the train data and tranforming it
train_scaled = sc.fit_transform(train)

In [ ]:
def process_data_by_timesteps(train_data, timesteps):
    
    print(f"Forming batches of data every {timesteps} days")
    
    X_train = []
    y_train = []

    ## Looping through the data
    for i in range(timesteps, 1913-timesteps):

        ### Getting a batch of timesteps
        X_train.append(train_data[i-timesteps:i])

        ### Setting the label as the next value in time after the timestamps
        y_train.append(train_data[i][0:30490]) 
    
    ## Viewing the shape of the first observation of the training data
    print(f"X_train shape: {X_train[0].shape}")
    print(f"y_train shape: {y_train[0].shape}")
    
    ## Viewing the shape after conversion
    print("Converting to arrays...")
    
    X_train = np.array(X_train, dtype = 'float16')
    y_train = np.array(y_train, dtype = 'float16')
    
    print(f"X_train array shape: {X_train.shape}")
    print(f"y_train array shape: {y_train.shape}")
    
    print("COMPLETE!!!")
    
    return X_train, y_train

In [ ]:
timesteps=28

In [ ]:
# Getting the processed train data

X_train_f, y_train_f = process_data_by_timesteps(train_data=train_scaled, timesteps=14)

In [ ]:
# Creating the model

## Tensorflow imports
import tensorflow as tf

## Setting the input shape
input_shape = (np.array(X_train_f).shape[1], np.array(X_train_f).shape[2])

## Creating a sequential model
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, input_shape=input_shape, return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(30490),
])

## Viewing the summary of the model
model.summary()

## Compiling the model
model.compile(optimizer='adam', metrics=['mse'], loss='mean_squared_error')

In [ ]:
# Creating a callback

callback = tf.keras.callbacks.EarlyStopping(monitor='mse', patience=3, min_delta=0.001)

In [ ]:
## Training/ Fitting the model

model.fit(X_train_f, y_train_f, epochs = 20, batch_size = 10, callbacks=[callback])

### Processing the test set

In [ ]:
def process_test_data_and_get_predictions(train_data, timesteps):
    
    # Extracting the first observation of test set by taking the last "timestamps" of the training data
    inputs = train_data[-timesteps:]
    
    # Transforming the same way as train data (NOT FITTING!)
    inputs = sc.transform(inputs)
    
    # Creating the test set
    X_test = []
    X_test.append(inputs[0:timesteps])
    X_test = np.array(X_test)
    
    # Using the trained model to predict 

    predictions = []

    ## Need to predict 28 days
    for j in range(timesteps,timesteps + 28):

        print(f"Observation Num: {j}")

        ### Predicting the next day
        forecast = model.predict(X_test[0,j - timesteps:j].reshape(1, timesteps, 30490))
        print(f"Forecast shape: {forecast.shape}")

        # Adding the combined data as the next batch
        X_test = np.append(X_test, forecast).reshape(1,j+1,30490)
        print(f"X_test shape: {X_test.shape}")

        # Inverse the scaling
        forecast = sc.inverse_transform(forecast)[:,0:30490]
        print(f"Forecast shape: {forecast.shape}")

        # Append the forecast into the predictions
        predictions.append(forecast)
    
    return predictions

In [ ]:
# Getting predictions

predictions = process_test_data_and_get_predictions(train, timesteps)

In [ ]:
import time

dataPath = '../input/m5-forecasting-accuracy/'

submission = pd.DataFrame(data=np.array(predictions).reshape(28,30490))

submission = submission.T
    
submission = pd.concat((submission, submission), ignore_index=True)

sample_submission = pd.read_csv(dataPath + "/sample_submission.csv")
    
idColumn = sample_submission[["id"]]
    
submission[["id"]] = idColumn  

cols = list(submission.columns)
cols = cols[-1:] + cols[:-1]
submission = submission[cols]

colsdeneme = ["id"] + [f"F{i}" for i in range (1,29)]

submission.columns = colsdeneme

currentDateTime = time.strftime("%d%m%Y_%H%M%S")

submission.to_csv("submission.csv", index=False)